In [1]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten

In [2]:
dataset_path = 'Images'

In [27]:
%matplotlib inline
import pandas as pd
import os,shutil,math,scipy,cv2
import numpy as np
import matplotlib.pyplot as plt
import random as rn


from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,roc_curve,auc

from PIL import Image
from PIL import Image as pil_image
from PIL import ImageDraw

from time import time
from glob import glob
from tqdm import tqdm
from IPython.display import SVG


from keras import backend as K
from keras.utils.np_utils import to_categorical
from keras import layers
from keras.utils.vis_utils import model_to_dot
from keras.applications.vgg16 import VGG16,preprocess_input
from keras.applications.xception import Xception
from keras.applications.nasnet import NASNetMobile
from keras.models import Sequential,Model
from keras.layers import Dense,Flatten,Dropout,Concatenate,GlobalAveragePooling2D,Lambda,ZeroPadding2D
from keras.layers import SeparableConv2D,BatchNormalization,MaxPooling2D,Conv2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,SGD
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger,ReduceLROnPlateau,LearningRateScheduler

In [4]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix

In [28]:
checkpoint_filepath = 'best_model.h5'
model_checkpoint = ModelCheckpoint(
    checkpoint_filepath,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

In [95]:
# Define the path to the images folder
data_dir = 'Images'
num_breeds = 30  # Change this to the desired number of folders

# Get a list of all subdirectories (dog breed folders) in the 'Images' directory
breed_folders = [folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))]

# Choose num_breeds random folders
selected_breeds = random.sample(breed_folders, num_breeds)

# Display the selected folders
print("Selected Breeds:")
print(selected_breeds)

Selected Breeds:
['n02088094-Afghan_hound', 'n02106030-collie', 'n02108551-Tibetan_mastiff', 'n02094114-Norfolk_terrier', 'n02106166-Border_collie', 'n02107908-Appenzeller', 'n02106382-Bouvier_des_Flandres', 'n02091635-otterhound', 'n02108089-boxer', 'n02097209-standard_schnauzer', 'n02104365-schipperke', 'n02106662-German_shepherd', 'n02113624-toy_poodle', 'n02111500-Great_Pyrenees', 'n02101556-clumber', 'n02105505-komondor', 'n02091032-Italian_greyhound', 'n02098413-Lhasa', 'n02090721-Irish_wolfhound', 'n02096051-Airedale', 'n02099429-curly-coated_retriever', 'n02097298-Scotch_terrier', 'n02086079-Pekinese', 'n02111277-Newfoundland', 'n02093754-Border_terrier', 'n02116738-African_hunting_dog', 'n02113978-Mexican_hairless', 'n02089867-Walker_hound', 'n02102318-cocker_spaniel', 'n02089078-black-and-tan_coonhound']


In [96]:
# Parameters
batch_size = 8
img_height, img_width = 150, 150
epochs = 10

In [97]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed=seed,
    classes=selected_breeds  # Pass the selected folders to the 'classes' parameter
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    seed=seed,
    classes=selected_breeds  # Pass the selected folders to the 'classes' parameter
)

Found 3519 images belonging to 30 classes.
Found 1487 images belonging to 30 classes.


In [98]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the convolutional base
for layer in base_model.layers[:15]:
    layer.trainable = False
for layer in base_model.layers[15:]:
    layer.trainable = True

# Build the model
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(num_breeds,activation='softmax'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 global_average_pooling2d_11  (None, 512)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_11 (Dropout)        (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 30)                15390     
                                                                 
Total params: 14,730,078
Trainable params: 7,094,814
Non-trainable params: 7,635,264
_________________________________________________________________


In [99]:
# Compile the model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.metrics import Precision, Recall

model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy', Precision(), Recall()])

#model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

#metrics_callback = MetricsCallback()

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
    #validation_steps = 1000,
    #steps_per_epoch = 1000,
    #callbacks=[metrics_callback]
)




Epoch 1/10
439/439 [==============================] - 94s 211ms/step - loss: 3.4163 - accuracy: 0.0362 - precision_22: 0.0000e+00 - recall_21: 0.0000e+00 - val_loss: 3.3831 - val_accuracy: 0.0534 - val_precision_22: 0.0000e+00 - val_recall_21: 0.0000e+00
Epoch 2/10
439/439 [==============================] - 60s 136ms/step - loss: 3.3856 - accuracy: 0.0544 - precision_22: 0.3333 - recall_21: 2.8482e-04 - val_loss: 3.3326 - val_accuracy: 0.0845 - val_precision_22: 0.0000e+00 - val_recall_21: 0.0000e+00
Epoch 3/10
439/439 [==============================] - 60s 136ms/step - loss: 3.2626 - accuracy: 0.0880 - precision_22: 0.4500 - recall_21: 0.0051 - val_loss: 3.0618 - val_accuracy: 0.1615 - val_precision_22: 0.3333 - val_recall_21: 6.7568e-04
Epoch 4/10
439/439 [==============================] - 61s 140ms/step - loss: 2.8707 - accuracy: 0.1834 - precision_22: 0.5516 - recall_21: 0.0396 - val_loss: 2.5086 - val_accuracy: 0.2682 - val_precision_22: 0.5957 - val_recall_21: 0.0946
Epoch 5/10
4